In [1]:
%pip install yandex_chain 
%pip install -U langchain-community
%pip install faiss-cpu

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
%pip uninstall jwt
%pip install PyJWT -U
%pip install cryptography

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
%pip install gradio

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [4]:
import requests
import json
import time
import jwt
import os
from yandex_chain import YandexLLM, YandexEmbeddings, YandexGPTModel
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
import faiss
import pickle
import pandas as pd
import gradio as gr

In [5]:
service_account_id = os.environ['service_account_id']
key_id = os.environ['key_id']
private_key = os.environ['my-API']

In [7]:
with open('/home/jupyter/datasphere/project/authorized_key.json', 'r') as f: 
  obj = f.read() 
  obj = json.loads(obj)
  private_key = obj['private_key']
  key_id = obj['id']
  service_account_id = obj['service_account_id']

now = int(time.time())
payload = {
        'aud': 'https://iam.api.cloud.yandex.net/iam/v1/tokens',
        'iss': service_account_id,
        'iat': now,
        'exp': now + 3600
      }

# Формирование JWT.
encoded_token = jwt.encode(
    payload,
    private_key,
    algorithm='PS256',
    headers={'kid': key_id}
  )

#Запись ключа в файл
with open('jwt_token.txt', 'w') as j:
   j.write(encoded_token) 
   
# Вывод в консоль
#print(encoded_token)


In [8]:
now = int(time.time())
payload = {
        'aud': 'https://iam.api.cloud.yandex.net/iam/v1/tokens',
        'iss': service_account_id,
        'iat': now,
        'exp': now + 360}

# Формирование JWT
encoded_token = jwt.encode(
    payload,
    private_key,
    algorithm='PS256',
    headers={'kid': key_id})

url = 'https://iam.api.cloud.yandex.net/iam/v1/tokens'
x = requests.post(url,  headers={'Content-Type': 'application/json'}, json = {'jwt': encoded_token}).json()
token = x['iamToken']

In [16]:
def load_retriever(n):
    # Определение имен файлов для загрузки
    if n == 0:
        combined_index_file = 'faiss_2016_index.index'
        combined_pkl_file = 'faiss_2016_index.pkl'
    else:
        combined_index_file = f'combined_{n}_index.index'
        combined_pkl_file = f'combined_{n}_index.pkl'
    
    # Загрузка индекса из файла
    index = faiss.read_index(combined_index_file)

    # Загрузка метаданных
    with open(combined_pkl_file, "rb") as f:
        vectorstore = pickle.load(f)

    # Обновление индекса в загруженном векторном хранилище
    vectorstore.index = index

    # Создание и возврат retriever
    return vectorstore.as_retriever()


In [17]:
def find_matching_indices(data, search_substring):
    matching_indice = 0
    
    for i, doc in enumerate(data):
        if search_substring in doc:
            matching_indice = i
    
    return matching_indice


In [18]:
def respond_to_query(year, query):
    year_to_n = {
        2016: 0,
        2017: 1,
        2018: 1,
        2019: 2,
        2020: 2,
        2021: 2,
        2022: 3,
        2023: 4
    }
    
    n = year_to_n[year] # Преобразование года в число от 0 до 4

    embeddings = YandexEmbeddings(folder_id=os.environ['folder_id'], api_key=os.environ['my-API'])
    retriever = load_retriever(n)

    template = """ 
    Коротко (не более 100 слов) ответь на вопрос, опираясь только на следующий контекст:{context}.
    Очень важно: к китайским именам и названиям добавляй их написание на китайском языке, а к японским именам и названиям добавляй их написание на японском языке. 
    Question: {question}.
    """
    prompt = ChatPromptTemplate.from_template(template)

    model = YandexLLM(folder_id=os.environ['folder_id'], api_key=os.environ['my-API'], model=YandexGPTModel.Pro)

    chain = (
        {"context": retriever, "question": RunnablePassthrough()} 
        | prompt 
        | model 
        | StrOutputParser()
    )
    data = pd.read_csv('/home/jupyter/datasphere/project/rag_east_data.csv')

    # Создадим пустой список для ссылок
    links = []
    results = retriever.invoke(query)
    # Проходим по каждому результату из 'results'
    for result in results:
        # Найдем все индексы, где результат есть в столбце 'translated_text'
        indices = find_matching_indices(data['translated_text'], str(result)[14:60])
        links.append(data['link'][indices])

    # Пример использования
    res = chain.invoke(query)
    final_result = add_sources(res, links)
    
    return final_result


In [19]:
def add_sources(res, sources):
    # Добавить отступ в одну строку
    result_with_sources = res + "\n\nВас могут заинтересовать следующие источники:\n\n"

    # Пронумерованные ссылки из списка
    for i, source in enumerate(sources, 1):
        result_with_sources += f"{i}.[{source}]({source})\n\n"
       

    return result_with_sources



In [ ]:
custom_css = """
<style>
    .container {
        max-width: 700px;
        margin: 0 auto;
    }
</style>
"""
with gr.Blocks() as demo:
    # Включаем стили
    gr.HTML(custom_css)
    
    # Разметка содержимого
    with gr.Column(elem_classes="container"):
        # Заголовок
        gr.Markdown("# Чатбот-консультант по восточным СМИ (команда 'Серсо')")
        
        # Выбор года от 2016 до 2023
        year = gr.Slider(2016, 2023, step=1, label="Выберите год от 2016 до 2023. База будет проанализирована по этот год включительно:", value=2016)
        
        # Ввод вопроса
        query = gr.Textbox(label="Введите ваш вопрос:")
        
        # Кнопка для отправки запроса
        answer_btn = gr.Button("Отправить запрос")
        
        # Место для отображения ответа
        output = gr.Markdown(label="Ответ бота:")

    # Соединение ввода, клика кнопки и выхода
    answer_btn.click(fn=respond_to_query, inputs=[year, query], outputs=output)

# Запуск приложения
demo.launch(share=True, debug=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://df10762dedf991cf2d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
